# Extract Regions Tensorflow Object Detection Inference


In [1]:
!pip install -U --pre tensorflow=="2.2.0"

     |████████████████████████████████| 516.2MB 31kB/s 
     |████████████████████████████████| 3.0MB 33.8MB/s 
     |████████████████████████████████| 460kB 49.3MB/s 
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


## Install Object Detection API

In [2]:
!git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2449, done.
remote: Counting objects: 100% (2449/2449), done.
remote: Compressing objects: 100% (2050/2050), done.
remote: Total 2449 (delta 588), reused 1356 (delta 372), pack-reused 0
Receiving objects: 100% (2449/2449), 30.85 MiB | 31.71 MiB/s, done.
Resolving deltas: 100% (588/588), done.


In [3]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp37-none-any.whl size=1624302 sha256=f2d5cf2cd7650ffc518b620cba0c4275d2c1e262f733354bb1ea0790a41f29f5
  Stored in directory: /tmp/pip-ephem-wheel-cache-s7w0n8dc/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
  Created wheel for avro-python3: filename=avro_python3-1.10.1-cp37-none-any.whl size=43735 sha256=16c08f13cea27653f24f4e4532adf67b31f2e974801151a8dcb9196841550863
  Stored in directory: /root/.cache/pip/wheels/65/fe/90/20d6d6d97223d80d20cb390be636619c536edab5658c12bdba
  Created wheel for dill: filename=dill-0.3.1.1-cp37-none-any.whl size=78532 sha256=efad99794ec0d12c2e6fbd8d097cd1bd3b5bef1a011ae19aa8a705f2e372115b
  Stored in directory: /root/.cache/pip/wheels/59/b1/91/f02e76c732915c4015ab4010f3015469866c1eb9b14058d8e7
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=270fefa66f664f000e849e71bf1cc3adbab4afc28b9dc0d7bd42

ERROR: multiprocess 0.70.11.1 has requirement dill>=0.3.3, but you'll have dill 0.3.1.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: apache-beam 2.28.0 has requirement avro-python3!=1.9.2,<1.10.0,>=1.8.1, but you'll have avro-python3 1.10.1 which is incompatible.


## Imports and setup

In [23]:
import os
import io
import matplotlib
import matplotlib.pyplot as plt
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity
import time
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import tensorflow as tf

from object_detection.utils import label_map_util, config_util
from object_detection.utils import visualization_utils as viz_utils

%matplotlib inline
# import warnings
# warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

## Connect with drive

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Load model

In [6]:
ROOT = 'gdrive/MyDrive/MIA2/landmark_detections'
PATH_TO_SAVED_MODEL = os.path.join(ROOT, 'models', 'saved_model')
PATH_TO_LABELS = os.path.join(ROOT, 'landmarks_label_map.pbtxt')

print('Model path {}'.format(PATH_TO_SAVED_MODEL))
print('Label path {}'.format(PATH_TO_LABELS))

Model path gdrive/MyDrive/MIA2/landmark_detections/models/saved_model
Label path gdrive/MyDrive/MIA2/landmark_detections/landmarks_label_map.pbtxt


In [7]:
# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)
print('Model loaded')

Model loaded


## Load label map

In [8]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)
category_index

{1: {'id': 1, 'name': 'mark'}}

## Run detections

In [9]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  return np.array(Image.open(path))

In [107]:
IMAGES_FOLDER = os.path.join(ROOT, 'data')
images_list = os.listdir(IMAGES_FOLDER)
# images_list = ['img_{}.jpg'.format(i) for i in [0]]
print('Images folder {}'.format(IMAGES_FOLDER))
print(images_list)

Images folder gdrive/MyDrive/MIA2/landmark_detections/data
['img_1914.jpg', 'img_150.jpg', 'img_2000.jpg', 'img_100.jpg', 'img_333.jpg', 'img_200.jpg', 'img_1950.jpg', 'img_1850.jpg', 'img_0.jpg', 'img_300.jpg', 'img_50.jpg', 'img_450.jpg', 'img_1900.jpg', 'img_350.jpg', 'img_400.jpg', 'img_250.jpg', 'img_57.jpg']


In [110]:
frames_tolerance = 5
min_score_thresh = 0.5
clusters_dict = {}
clusters_lst = []

for idx, image_name in enumerate(images_list):
  print('{}/{}. Running inference for {}'.format(idx + 1, len(images_list), image_name))
  image_np = load_image_into_numpy_array(os.path.join(IMAGES_FOLDER, image_name))
  input_tensor = tf.convert_to_tensor(image_np)
  input_tensor = input_tensor[tf.newaxis, ...]

  # run detections
  detections = detect_fn(input_tensor)

  num_detections = int(detections.pop('num_detections'))

  detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
  detections['num_detections'] = num_detections
  
  # detection_classes should be ints
  detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

  draw_detections = False
  if draw_detections:
    image_np_with_detections = image_np.copy()
    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],
        detections['detection_classes'],
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=20,
        min_score_thresh=min_score_thresh,
        agnostic_mode=False)
    plt.figure(figsize=(16, 20))
    plt.imshow(image_np_with_detections)

  h, w, _ = image_np.shape
  y_ = []

  for idx_, score in enumerate(detections['detection_scores']):
    if score >= min_score_thresh:
      ymin, xmin, ymax, xmax = detections['detection_boxes'][idx_]
      ymin, xmin, ymax, xmax = int(ymin * h), int(xmin * w), int(ymax * h), int(xmax * w)
      y_.append((ymin + ymax) / 2)

  best_n_clusters = None
  cluster_score = None
  try:
    for idx_c, n_clusters in enumerate(range(2, 4)):
      kmeans = KMeans(n_clusters=n_clusters)
      kmeans.fit(np.array(y_).reshape(-1, 1))
      silhouette_avg = silhouette_score(np.array(y_).reshape(-1, 1), kmeans.predict(np.array(y_).reshape(-1, 1)))
      if idx_c == 0 or cluster_score < silhouette_avg:
        best_n_clusters = n_clusters
        cluster_score = silhouette_avg
  except :
    pass
  
  lines = []
  if best_n_clusters is None:
    print('Not possible to create clusters')
  else:
    kmeans = KMeans(n_clusters=best_n_clusters)
    kmeans.fit(np.array(y_).reshape(-1, 1))

    cnt_dict = {}
    counter = np.unique(kmeans.labels_, return_counts=True)
    for key, ctr in zip(counter[0], counter[1]):
      cnt_dict[key] = ctr
    
    for key in cnt_dict.keys():
      if cnt_dict[key] > 1:
        lines.append(int(kmeans.cluster_centers_.reshape(-1)[key]))

  draw_lines = False
  if draw_lines:
    img = Image.fromarray(image_np_with_detections, 'RGB')
    draw = ImageDraw.Draw(img)
    for limit in lines:
      draw.line([(0, limit), (w, limit)], width=10, fill='red')
    plt.figure(figsize=(16, 20))
    plt.imshow(img)
  lines.sort()
  # print('Clusters:', lines)

  clusters_lst.append([image_name, len(lines), lines])
  # analyze only one image
  # break

1/17. Running inference for img_1914.jpg
Not possible to create clusters
2/17. Running inference for img_150.jpg
3/17. Running inference for img_2000.jpg
4/17. Running inference for img_100.jpg
5/17. Running inference for img_333.jpg
6/17. Running inference for img_200.jpg
7/17. Running inference for img_1950.jpg
8/17. Running inference for img_1850.jpg
9/17. Running inference for img_0.jpg
10/17. Running inference for img_300.jpg
11/17. Running inference for img_50.jpg
12/17. Running inference for img_450.jpg
13/17. Running inference for img_1900.jpg
14/17. Running inference for img_350.jpg
15/17. Running inference for img_400.jpg
16/17. Running inference for img_250.jpg
17/17. Running inference for img_57.jpg


In [111]:
df = pd.DataFrame(clusters_lst)
df.columns=['IMG', 'n_clusters', 'Lines']
df = df.set_index('IMG')
df

,n_clusters,Lines
IMG,,
img_1914.jpg,0,[]
img_150.jpg,3,"[242, 1346, 2086]"
img_2000.jpg,2,"[66, 1427]"
img_100.jpg,2,"[400, 1565]"
img_333.jpg,2,"[209, 1807]"
img_200.jpg,3,"[151, 773, 1750]"
img_1950.jpg,2,"[63, 907]"
img_1850.jpg,2,"[330, 1237]"
img_0.jpg,3,"[287, 1114, 2015]"


In [112]:
a = df.index.to_series().str.rsplit('.jpg').str[0].str.rsplit('_').str[-1].astype(int).sort_values()
df = df.reindex(index=a.index)
df

,n_clusters,Lines
IMG,,
img_0.jpg,3,"[287, 1114, 2015]"
img_50.jpg,3,"[482, 1421, 2092]"
img_57.jpg,2,"[595, 1548]"
img_100.jpg,2,"[400, 1565]"
img_150.jpg,3,"[242, 1346, 2086]"
img_200.jpg,3,"[151, 773, 1750]"
img_250.jpg,3,"[41, 589, 1680]"
img_300.jpg,2,"[457, 1523]"
img_333.jpg,2,"[209, 1807]"
